# 神经网络  
普通神经网络：
<img src="images/v2-c705d5552a53e7f98a5db1051b4b1ecf_b.png">  
每一层网络所做的就是 y=W*X+b，权重W的维数由X和输出输出决定，比如X是10维向量，想要输出的维数，也就是中间层的神经元个数为20，那么W的维数就是20×10，b的维数就是20×1，这样输出的y的维数就为20。  

中间层的维数可以自己设计，而最后一层输出的维数就是你的分类数目，比如我们等会儿要做的MNIST数据集是10个数字的分类，那么最后输出层的神经元就为10。  

* **Code** 

In [ ]:
# 定义模型
class Neuralnetwork(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(Neuralnetwork, self).__init__()
        self.layer1 = nn.Linear(in_dim, n_hidden_1)
        self.layer2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.layer3 = nn.Linear(n_hidden_2, out_dim)
 
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x
 
model = Neuralnetwork(28*28, 300, 100, 10)
if torch.cuda.is_available():
    model = model.cuda()
 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

上面定义了三层神经网络，输入是28×28，因为图片大小是28×28，中间两个隐藏层大小分别是300和100，最后是个10分类问题，所以输出层为10.

训练过程与之前完全一样，在此再重复了，可以直接去[github](https://github.com/L1aoXingyu/pytorch-beginner)上参看完整的代码。  

这是50次之后的输出结果，可以和上一节logistic回归比较一下  
<img src="images/v2-2436da353dce8bbeef592ac7670c74f8_b.png">  
可以和上一节logistic回归比较, 可以发现准确率大大提高，其实logistic回归可以看成简单的一层网络，从这里我们就可以看出为什么多层网络比单层网络的效果要好，这也是为什么深度学习要叫深度的原因。

## 卷积神经网络  
卷积神经网络是一类专门用来处理具有类似网络结构的数据的神经网络。  
$$s(t)=\int x(a)w(t-a)da$$  
离散形式为：  
$$S(i,j)=(I*K)(i,j)=\sum_m\sum_nI(i+m,j+n)K(m,n)$$  
其中K为卷积核。

实例：识别“x”和“o”：  
<img src="images/23423423234343434.jpg" width="50%">  
计算机对于图像的认知是在矩阵上的，每一张图片有rgb二维矩阵（不考虑透明度），每一个像素有三个rgb数字构成，这里只有黑白两种颜色，就只用1和-1来表示：  
<img src="images/4576578922111122.jpg" width="50%">  
卷积核从左往右，依次和图像作用，一层卷积操作完成之后得到新的数据作为下一层的输入：  
<img src="images/4445454666667799.jpg" width="50%"> 

* 池化层  

在图像经过以上的卷积层后，得到了一个新的矩阵，而矩阵的大小，则取决于卷积核的大小，和边缘的填充方式。通常卷积层之后会接一个池化层。池化就是缩减图像尺寸和像素关联性的操作，只保留我们感兴趣（对于分类有意义）的信息，常用的就是2x2的最大池化：  
<img src="images/1234565879806.jpg" width="50%">  

* 全连接层  

全连接层中的每个神经元与其前一层的所有神经元进行全连接．全连接层可以整合卷积层或者池化层中具有类别区分性的局部信息．为了提升 CNN网络性能，全连接层每个神经元的激励函数一般采用ReLU函数。最后一层全连接层的输出值被传递给一个输出，可以采用softmax逻辑回归（softmax regression）进行 分 类，该层也可 称为 softmax层（softmax layer）．  
<img src="images/v2-6840898a5893c31f77c89bcd539b4922_hd.jpg" width="50%">  

* **Code**  

In [ ]:
# 定义 Convolution Network 模型
class Cnn(nn.Module):
    def __init__(self, in_dim, n_class):
        super(Cnn, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_dim, 6, 3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5, stride=1, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
        )
 
        self.fc = nn.Sequential(
            nn.Linear(400, 120),
            nn.Linear(120, 84),
            nn.Linear(84, n_class)
        )
 
    def forward(self, x):
        out = self.conv(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
 
model = Cnn(1, 10)  # 图片大小是28x28
use_gpu = torch.cuda.is_available()  # 判断是否有GPU加速
if use_gpu:
    model = model.cuda()
# 定义loss和optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

这是其中一种搭建神经网络的方式，其它方式可见[https://www.cnblogs.com/denny402/p/7593301.html](https://www.cnblogs.com/denny402/p/7593301.html)。

## 循环神经网络  
循环神经网络是一类用于处理序列数据的神经网络。正如卷积网络可以很容易地扩展到具有很大宽度和高度的图像，以及处理大小可变的图像，循环网络可以款站到更长的序列。大多数循环网络也能处理可变长度的序列。  
<img src="images/v2-7253059042053ed3185689162fe3842f_b.png" >  

* **LSTM(Long short-term memory,长短期记忆）**  

LSTM模型是一个向后传递的链式模型，而现在广泛使用的RNN其实就是LSTM，序列中每个数据传入LSTM可以得到两个输出，而这两个输出和序列中下一个数据一起又作为传入LSTM的输入，然后不断地循环向后，直到序列结束。如图：  
<img src="images/v2-4186280fa280fe29ab069588550b6b33_b.png">  

其中包括：
1，$f_t$为遗忘门：
<img src="images/v2-5e3055ec3b7f87d0bfb0a93558d4dd96_b.png">  

2、3，$i_t$为输入门、$\tilde C_t$被称为状态：  
<img src="images/v2-836ada2c436175835e1fdc6d8bc8a9b5_b.png">   

4，$C_t$为几条线路的叠加：  
<img src="images/v2-b18bb5c37aa78e8d48598b92a0a1b10f_b.png">   

5，$o_t$为输出门，$h_t$为输出。
<img src="images/v2-0bf555937d5121cd0653e726a5670aa0_b.png"> 

* **Code**

In [ ]:
class Rnn(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_layer, n_class):
        super(Rnn, self).__init__()
        self.n_layer = n_layer
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(in_dim, hidden_dim, n_layer,
                            batch_first=True)
        self.classifier = nn.Linear(hidden_dim, n_class)
 
    def forward(self, x):
        # h0 = Variable(torch.zeros(self.n_layer, x.size(1),
                                #   self.hidden_dim)).cuda()
        # c0 = Variable(torch.zeros(self.n_layer, x.size(1),
                                #   self.hidden_dim)).cuda()
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.classifier(out)
        return out
 
model = Rnn(28, 128, 2, 10)  # 图片大小是28x28
use_gpu = torch.cuda.is_available()  # 判断是否有GPU加速
if use_gpu:
    model = model.cuda()
# 定义loss和optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

这里定义了一个LSTM模型，我们需要传入的参数是输入数据的维数28，LSTM输出的维数128，LSTM网络层数2层以及输出的类数10。

* **train**  
用MNIST训练，训练过程的batch_size设置为100，learning_rate设置为0.01，训练20次，最后得到的结果如下：  
<img src="images/v2-b77b8587c7755cfd298b0e0d46ed6c08_b.png">  
可以发现对于简单的图像分类RNN也能得到一个较好的结果，虽然CNN更多的用在图像领域而RNN更多的用在自然语言处理中。